In [7]:
import cv2
import numpy as np
import copy
import math
import os
import keras
import tensorflow as tf
import imutils
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

# 1
# Create a setup screen, showing hand on screen to set the right threshold?
# User can move sliders to set the correct threshold
# Include blown out hands

# 2
# Remove face in video?

# 3
# Multithreading for better performance

# 4
# Write about smoothing 

In [8]:
def loadModel():
    
    version = 0
    modelDir = "Model Versions/HandModelV"

    # This method always gets the most up to date model.
    
    while True:
        try:
            version = version + 1
            f = open(modelDir + str(version) + ".h5", 'r')
            f.close()
        except:
            break
            
    model = keras.models.load_model(modelDir + str(version-1) + ".h5")
    
    print("Using model " + modelDir + str(version-1) + ".h5")
    
    return model

model = loadModel()

Using model Model Versions/HandModelV1.h5


In [9]:
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

version = 1
dir = "../../../../HandsData/OpenCVHandsData/One/Me/Second/2one"

def saveImage(handBox):
                
    global version
        
    while True:
        try:
            f = open(dir + str(version) + ".jpg", 'r')
            f.close()
            version = version + 1
        except:
            break

    try:
        cv2.imwrite(dir + 1 + ".jpg", handBox)
    except:
        pass
    

In [10]:
sizeX = 60
sizeY = 100

def resize(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    h, w, ch = image.shape
    
    zeros = np.zeros((sizeY, sizeX, 3))
    
    tempImage = image.copy()
    
    #Apparently there's a bug in OpenCV that causes the program to crash if the resized image is too large.
    
    image = imutils.resize(tempImage, height=sizeY)
    if image.shape[1] > sizeX:
        image = imutils.resize(tempImage, width=sizeX)

    zeros[:image.shape[0], :image.shape[1]] = image    
    
    return zeros

In [11]:
outputText = ""
combinedOutput = []

def classifyHand(handBox):
    
    global outputText, combinedOutput
    
    try :
        handBox = resize(handBox)
    except:
        print("Mitigating error") # (Nobody will notice :) )
        return outputText, combinedOutput
    
#     plt.imshow(handBox)
#     plt.show()
    
    handBox = np.array(handBox)
        
    pred_hot = model.predict(np.expand_dims(handBox, axis=0))[0]

    
    if pred_hot[0] > pred_hot[1]:
        outputText = "Index " + str(pred_hot[0])
    else:
        outputText = "Fist " + str(pred_hot[1])
    
    
    combinedOutput = ["Fist: " + str(pred_hot[1]), "Index " + str(pred_hot[0])]
    
    return outputText, combinedOutput

In [12]:
camera = cv2.VideoCapture(0)

while (True):

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = camera.read()

    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # Smoothing

#     cv2.imshow("Output", frame)

    # Removing background

    background = cv2.createBackgroundSubtractorMOG2(0,50)

    kernel = np.ones((3,3), np.uint8)

    bgMask = background.apply(frame)
    bgMask = cv2.erode(bgMask, kernel, iterations=1)

    mask = cv2.bitwise_and(frame, frame, mask = bgMask)
    
    
    # Detecting skin

    hsv = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    skinMask = cv2.inRange(hsv, lower, upper)

    #cv2.imshow("Threshold", skinMask)

    # Contours

#     skinMask = skinMask.copy()

    contours, heirarchy = cv2.findContours(skinMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        maxArea = -1
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if area > maxArea:
                maxArea = area
                res = contours[i]
    
        hull = cv2.convexHull(res)
        drawing = np.zeros(mask.shape, np.uint8)
        #res is outline of hand
        cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)
        #hull is box around hand
        cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)
        
        x, y, w, h = cv2.boundingRect(res)


        

        
        try:
            
            # Adding padding around the box of interest. If the hand is at the edge of the screen this won't work.
            
            handBox = frame[int(y-h/4):int(y+h*1.25), int(x-w/4):int(x+w*1.25)]
            
            
        except:
            
            handBox = frame[y:y+h, x:x+w]
            
        
        try:
            cv2.imshow("Box", handBox)
        except:
            print("Too close")
        
        
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # saveImage is used for collecting images for training the classifier

        #saveImage(handBox)

        
        cv2.rectangle(frame, (int(x-w/4), int(y-h/4)), (int(x+w*1.25), int(y+h*1.25)), (0, 0, 255), 2)
        

        #print(model.predict(np.expand_dims(resize(handBox), axis=0))[0])
                
        
        outputText, combinedOutput = classifyHand(handBox)
        
        spacing = 25
        for i in range(len(combinedOutput)):
            cv2.putText(frame, combinedOutput[i], (10, 25 + (i * spacing)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.putText(frame, outputText, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.imshow("Output", frame)
    
    else:
        
        cv2.imshow("Output", frame)

        
    
camera.release()
cv2.destroyAllWindows()